In [1]:
from methods.llm import get_async_vllm_endpoint
import os 

# Unlimited LLM endpoints
endpoint_id = "vllm-8sz1f7zg7oy0ui"
api_key = "rpa_EPOJED42G59S80Y6SKMCOI330EQU4JPPMKV2UD2W7j0uku"
get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key)

Could not load vllm class, check CUDA support and GPU RAM size


#### Temasek Foundation Dataset

In [3]:
# 1. Test cases (dataloading ...)
import sys 
sys.path.append("../notebook/")
from optm.soft_prompt import load_tf_data

train_data, test_data = load_tf_data()

# reference metric fuction below ... we care about prediction of label, specifically "no"
# we care about precision more than we care abou recall on 'No' prediction 

# Node dataset should ideally be concise and pure application info included ...
from methods.meta_prompt import MetaPrompt, PromptMode

tf_meta_prompt = MetaPrompt(
    task = "Evaluate grant application, make a decision (Yes, No, Maybe) and a brief comment explanating your decision on why this project is likely to be accepted or rejected.",
    func_name = "evaluate_grant",
    inputs = ["project_description"],
    outputs = ["label", "comment"],
    input_types = ["str"],
    output_types = ["str", "str"],
    mode = PromptMode.PROMPT
)


test_cases = []


In [7]:
train_data["prompt"][0]

'Given the following details of a grant application, determine the likelihood of acceptance. \n    Assess how well the project aligns with the challenge theme, its viability, its potential impact, and any competitive advantage. \n    Consider the stage of development, the proof of concept status, and existing partnerships. \n    Provide your decision (Yes, No, Maybe) and a brief comment explanating your decision on why this project is likely to be accepted or rejected.\n\n    Grant Application Summary:\n    - **Project Theme:** Food Systems & Sustainable Agriculture\n    - **Category:** Food security\n    - **Subcategory:** Food Waste Management\n    - **Project Title:** You are not alone - Human is near you\n    - **Grant Amount Sought:** None\n    - **Objectives:** None\n    - **Have you obtained proof of concept for your project?** None\n\n    Solution Summary: This application, "You are not alone - Human is near you," aims to address the issue of malnutrition in Gujarat, India, par

In [2]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.evolnode import EvolNode
from methods.llm import get_groq_response, get_claude_response

# Code + Compilor Task
# mp = MetaPrompt("Search for age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.CODE)
# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

test_inputs = [c[0] for c in test_cases]

node = EvolNode(mp, None, None, get_response=get_endpoint_response, test_cases=test_cases) # setting manual test cases

node.evolve("i1", replace=True, batch_size=20, num_runs=2, print_summary=True) # Scale up batch size


input_dict = {"name": "Dilireba"}
output_dict = node(input_dict, max_attempts=6) # Batch Inference with vLLM

# node.get_response = get_groq_response # fast sequential inference
# output_dict = node(input_dict, max_attempts=6, batch_inference=False)
 
print("Output dict: ", output_dict)

Processing LLM queries: 100%|██████████| 20/20 [00:11<00:00,  1.70it/s]


 :: Total time elapsed: 11.79s, 0 errors


Processing LLM queries: 100%|██████████| 24/24 [00:39<00:00,  1.66s/it]


 :: Total time elapsed: 39.80s, 0 errors


Processing LLM queries: 0it [00:00, ?it/s]


 :: Total time elapsed: 0.00s, 0 errors
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 0.33
  🎯 Functional fitness: 1.00
  ⭐ Global fitness:     0.67
  🔄 Compiled solutions:        16
  ⏱️ Time breakdown:
     :: Query time: 3.74s
     :: Evolution time: 11.84s
     :: Evaluation time: 73.45s
     :: Total time: 89.03s


📊 Code 0: Fitness: 25.0%
--------------------------------------------------------------------------------
❌ Error Messages:
Input: {'name': 'ChengXiao'}, prediction is not aligned with expected output, Expected: {'age': 26} Predicted: {'name': 'ChengXiao', 'age': 25, ' occupation': 'Artist'}, Error message: 
Value mismatch for key age: 25 != 26


Failed to parse LLM response -- JsonDecodeError : 
Expecting value: line 1 column 1 (char 0)AstLiteralError : 
invalid syntax (<unknown>, line 1)


📊 Code 1: Fitness: 8.3%
--------------------------------------------------------------------------------
❌ Error Messages:
Input: {'name': 'Dilireba'}, prediction is not

Processing LLM queries: 100%|██████████| 6/6 [00:02<00:00,  2.10it/s]

 :: Total time elapsed: 2.86s, 0 errors
Output dict:  {'age': 32}
